In [1]:
import pandas as pd
import tqdm

In [2]:
metadata_path = './data/abo-images-small/images/metadata/images.csv.gz'
listings_path = './data/abo-listings/listings/metadata/'

In [3]:
metadata_df = pd.read_csv(metadata_path, compression='gzip')

In [4]:
listings_df = pd.DataFrame()

for i in tqdm.tqdm(range(10)):
    temp_df = pd.read_json(f'{listings_path}listings_{i}.json.gz', lines=True, compression='gzip')
    listings_df = pd.concat([listings_df, temp_df], ignore_index=True)

for i in tqdm.tqdm(['a', 'b', 'c', 'd', 'e', 'f']):
    temp_df = pd.read_json(f'{listings_path}listings_{i}.json.gz', lines=True, compression='gzip')
    listings_df = pd.concat([listings_df, temp_df], ignore_index=True)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:17<00:00,  2.91s/it]


In [5]:
listings_df.dtypes

brand                  object
bullet_point           object
color                  object
item_id                object
item_name              object
model_name             object
model_number           object
model_year             object
product_type           object
style                  object
main_image_id          object
other_image_id         object
item_keywords          object
country                object
marketplace            object
domain_name            object
node                   object
item_dimensions        object
item_weight            object
material               object
fabric_type            object
color_code             object
product_description    object
spin_id                object
3dmodel_id             object
pattern                object
finish_type            object
item_shape             object
dtype: object

In [6]:
listings_df.dtypes

brand                  object
bullet_point           object
color                  object
item_id                object
item_name              object
model_name             object
model_number           object
model_year             object
product_type           object
style                  object
main_image_id          object
other_image_id         object
item_keywords          object
country                object
marketplace            object
domain_name            object
node                   object
item_dimensions        object
item_weight            object
material               object
fabric_type            object
color_code             object
product_description    object
spin_id                object
3dmodel_id             object
pattern                object
finish_type            object
item_shape             object
dtype: object

In [7]:
# drop unnecessary columns

filtered_df = listings_df[['item_name', 'main_image_id', 'country', 'node',]]

In [8]:
# filter out any non US products

filtered_df = filtered_df[filtered_df['country'] == 'US']

In [9]:
# remove unnecessary metadata and store only string value title and categories

def extract_node_name(nodes):
    try:
        if nodes and isinstance(nodes, list):  # Check if 'nodes' is not None or empty, and is a list
            if isinstance(nodes[0], dict) and 'node_name' in nodes[0]:  # Check if the first item is a dictionary and has 'node_name'
                return nodes[0]['node_name']
    except Exception as e:
        print(f"Error: {e}")  # Print any exceptions that occur to help with debugging
    return None  # Return None if any conditions fail

def get_en_us_name(item_names):
    for item in item_names:
        if item['language_tag'] == 'en_US':
            return item['value']
    return None  # Return None  if 'en_US' is not found

filtered_df['node_name'] = filtered_df['node'].apply(extract_node_name)
filtered_df['product_name'] = filtered_df['item_name'].apply(get_en_us_name)
filtered_df = filtered_df.drop(columns=['node', 'country', 'item_name'])

# Remove rows where 'node_name' is either None or an empty string
filtered_df = filtered_df[filtered_df['node_name'].notna() & (filtered_df['node_name'] != '')]

In [10]:
# filtered_df.to_excel('filtered_abo_listings.xlsx')

In [11]:
metadata_df.to_excel('product_metadata.xlsx')